This program implements a fair solution of readers and writers in Java using threads and semaphores. The program starts with one reader and one writer. The writer generates in 1 second intervals from 0 .. `numIters` (command line argument), starting from the middle and going up. The writer is given preference, so if there are multiple readers, all readers get updates from single writers. The program supports multiple readers and multiple writers.

In [2]:
%%writefile ReaderWriter.java
import java.util.concurrent.Semaphore;
import java.lang.String;

public class ReaderWriter {
    /* global variables and semaphores */
    public static Semaphore e = new Semaphore(1), r = new Semaphore(0), w = new Semaphore(0);
    public static volatile int nr = 0, nw = 0, dr = 0, dw = 0;
    public static volatile int data = 0, maxdata = 0;
    public static int numIters = 0;

    static void reader() {
        maxdata = 0;
        System.out.println("Reader starting");
        while (true) {
            /* entry protocol */
            try {e.acquire();} catch(Exception e) {}
            if (nw > 0 || dw > 0) {
                dr += 1; e.release(); 
                try {r.acquire();} catch(Exception e) {}
            }
            nr += 1;
            if (dr > 0) {
                dr -= 1; r.release();
            } else {
                e.release();
            }
            
            if (data > maxdata) System.out.printf("Reader in critical section read %d\n", data);
            maxdata = data > maxdata ? data : maxdata;
            
            /* exit protocol */
            try {e.acquire();} catch(Exception e) {}
            nr -= 1;
            if (nr == 0 && dw > 0) {
                dw -= 1; w.release();
            } else {
                e.release();
            }
        }
    }

    static void writer() {
        System.out.printf("Writer starting\n");
        for (int i = 0; i < numIters; i++) {
            /* entry protocol */    
            try {e.acquire();} catch(Exception e) {}
            if (nr > 0 || nw > 0) {
                dw += 1; e.release(); 
                try {w.acquire();} catch(Exception e) {}
            }
            nw += 1;
            e.release();
            
            System.out.printf("Writer in critical section\n");
            data = (i + numIters / 2) % numIters;
            System.out.printf("Writer writing %d\n", data);
            
            /* exit protocol */
            try {e.acquire();} catch(Exception e) {}
            nw -= 1;
            if (dw > 0) {
                dw -= 1; w.release();
            } else if (dr > 0) {
                dr -= 1; r.release();
            } else {
                e.release();
            }
            
            try {Thread.sleep(1000);} catch (Exception e) {}; // sleep 1 sec
        }
    }
    public static void main(String args[]) {
        numIters = Integer.parseInt(args[0]);
        Thread r = new Thread() {{setDaemon(true);} public void run() {reader();}};
        Thread w = new Thread() {public void run() {writer();}};

        r.start(); w.start();
        try {w.join();} catch (Exception e) {}
        System.out.printf("Max data %d\n", maxdata);       
    }
}

Overwriting ReaderWriter.java


In [3]:
!javac ReaderWriter.java

In [4]:
!java ReaderWriter 5

Writer starting
Reader starting
Writer in critical section
Writer writing 2
Reader in critical section read 2
Writer in critical section
Writer writing 3
Reader in critical section read 3
Writer in critical section
Writer writing 4
Reader in critical section read 4
Writer in critical section
Writer writing 0
Writer in critical section
Writer writing 1
Max data 4


In [5]:
!java ReaderWriter 10

Writer starting
Writer in critical section
Writer writing 5
Reader starting
Reader in critical section read 5
Writer in critical section
Writer writing 6
Reader in critical section read 6
Writer in critical section
Writer writing 7
Reader in critical section read 7
Writer in critical section
Writer writing 8
Reader in critical section read 8
Writer in critical section
Writer writing 9
Reader in critical section read 9
Writer in critical section
Writer writing 0
Writer in critical section
Writer writing 1
Writer in critical section
Writer writing 2
Writer in critical section
Writer writing 3
Writer in critical section
Writer writing 4
Max data 9
